# Imports

In [1]:
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np

import sys
sys.path.insert(0, "lib/")
from data.coco_dataset import CocoDataset
from utils.preprocess_sample import preprocess_sample
from utils.collate_custom import collate_custom
from utils.utils import to_cuda_variable
import utils.result_utils as result_utils
from utils.json_dataset_evaluator import evaluate_boxes
from model.detector import detector

torch_ver = torch.__version__[:3]

# Parameters

In [2]:
# Pretrained model
# https://s3-us-west-2.amazonaws.com/detectron/35857281/12_2017_baselines/e2e_faster_rcnn_R-50-C4_2x.yaml.01_34_56.ScPH0Z4r/output/train/coco_2014_train%3Acoco_2014_valminusminival/generalized_rcnn/model_final.pkl
arch='resnet50'
pretrained_model_file = 'files/trained_models/faster/model_final.pkl'

# COCO minival2014 dataset path
coco_ann_file='datasets/data/coco/annotations/instances_minival2014.json'
img_dir='datasets/data/coco/val2014'

# Create dataset

In [3]:
dataset = CocoDataset(ann_file=coco_ann_file,img_dir=img_dir,
                       sample_transform=preprocess_sample(target_sizes=[800]))
dataloader = DataLoader(dataset, batch_size=1, # only batch_size=1 is supported by now
                        shuffle=False, num_workers=0, collate_fn=collate_custom)

loading annotations into memory...
Done (t=1.38s)
creating index...
index created!


# Create detector model

In [4]:
model = detector(arch=arch,
                 detector_pkl_file=pretrained_model_file,
                 use_rpn_head = True)
model = model.cuda()

loading pretrained weights


# Evaluate

In [5]:
# Create data structure to store results
all_boxes, all_segms, all_keyps = result_utils.empty_results(dataset.num_classes, len(dataset)) 
# (only all_boxes will be used for fast RCNN)

In [ ]:
# Compute detections for whole dataset
for i, batch in enumerate(dataloader):
    batch = to_cuda_variable(batch)
    # forward pass
    if torch_ver=="0.4": # handle change in "volatile"
        with torch.no_grad():
            class_scores,bbox_deltas,rois,_=model(batch['image'],
                                                  scaling_factor=batch['scaling_factors'])   
    else:
        class_scores,bbox_deltas,rois,_=model(batch['image'],
                                              scaling_factor=batch['scaling_factors'])   
    # postprocess output:
    # - convert coordinates back to original image size, 
    # - treshold proposals based on score,
    # - do NMS.
    scores_final, boxes_final, boxes_per_class = result_utils.postprocess_output(rois,
                                                                    batch['scaling_factors'],
                                                                    batch['original_im_size'],
                                                                    class_scores,
                                                                    bbox_deltas)
    # store results
    result_utils.extend_results(i, all_boxes, boxes_per_class)
    
    if i%100==0:
        print("{}/{}".format(i+1,len(dataset)))
        
print('Done!')

1/5000
101/5000
201/5000
301/5000
401/5000
501/5000
601/5000
701/5000
801/5000
901/5000
1001/5000
1101/5000
1201/5000
1301/5000
1401/5000
1501/5000
1601/5000


In [11]:
# Save detection results
np.save('files/results/all_boxes_faster.npy',all_boxes)

In [12]:
# Compute evaluation metrics
coco_eval = evaluate_boxes(json_dataset=dataset.coco, 
                           all_boxes=all_boxes, 
                           output_dir='files/results/',
                           use_salt=False, cleanup=False)

Loading and preparing results...
DONE (t=1.97s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=40.78s).
Accumulating evaluation results...
DONE (t=6.83s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.365
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.573
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.393
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.184
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.406
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.506
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.308
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.474
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.492
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1